In [9]:
import re
import os
import pandas as pd

def findAllFiles(path, extension):
    filenames = os.listdir(path)
    return [ os.path.join(path,filename) for filename in filenames if filename.endswith( extension ) ]

def findAllFileNames():
    cur_path = os.getcwd()
    relative_path = 'redditWSB-main\data\\reddit\daywise'
    path = os.path.join(cur_path,relative_path)
    extension =".csv"
    file_lists = findAllFiles(path,extension)
    return file_lists

def parseCandidates(str):
    if not str: return []
    tickers_3 = re.findall('([A-Z]{3})',str)
    tickers_4 = re.findall('([A-Z]{4})',str)
    return tickers_3+tickers_4

def getCandidates(df,col_name):
    tmp = df[col_name].dropna().apply(parseCandidates)
    tmp = [i for i in tmp if len(i) > 0]
    stock_list = []
    for i in tmp:
        stock_list.extend(i)
    stock_list = list(dict.fromkeys(stock_list))
    return stock_list

def getAllCandidates(filename):
    df = pd.read_csv(filename)
    stock_list_title = getCandidates(df,'title')
    stock_list_selftext = getCandidates(df,'selftext')
    stock_list = list(dict.fromkeys(stock_list_title+stock_list_selftext))
    return stock_list

In [10]:
file_list = findAllFileNames()
complete_ticker_list = []
for i in file_list:
    stock_list = getAllCandidates(i)
    complete_ticker_list.append(stock_list)
complete_ticker_list

[['LIT',
  'PLT',
  'TSL',
  'FUT',
  'PLTR',
  'TSLA',
  'FUTU',
  'GME',
  'SPY',
  'WSB',
  'WSBV',
  'MGM',
  'NSA',
  'BNG',
  'BNGO',
  'THA',
  'YOU',
  'WAL',
  'LST',
  'REE',
  'TBE',
  'FOR',
  'THI',
  'AMA',
  'ZIN',
  'YEA',
  'THAN',
  'WALL',
  'STRE',
  'ETBE',
  'THIS',
  'AMAZ',
  'YEAR',
  'FSD',
  'NPA',
  'CAL',
  'CALL',
  'AST',
  'AOC',
  'YOL',
  'TRA',
  'DES',
  'WOR',
  'YOLO',
  'TRAD',
  'WORK',
  'STO',
  'POS',
  'TIN',
  'STOP',
  'POST',
  'AMC',
  'BUL',
  'NKS',
  'BULL',
  'STON',
  'JUI',
  'ICR',
  'JUIC',
  'ENP',
  'ENPH',
  'BAB',
  'BABA',
  'XOM',
  'FTW',
  'AMD',
  'NIO',
  'NYS',
  'NYSE',
  'SPA',
  'SPAC',
  'MIG',
  'CON',
  'DRO',
  'PPI',
  'MIGH',
  'CONT',
  'INUE',
  'DROP',
  'PING',
  'ETF',
  'VLD',
  'VLDR',
  'ASA',
  'ASAP',
  'PBW',
  'DNN',
  'GLU',
  'GLUE',
  'AAP',
  'DRI',
  'AAPL',
  'DRIP',
  'USA',
  'USAA',
  'SEC',
  'MVP',
  'PLE',
  'ASE',
  'HEL',
  'WIT',
  'OPT',
  'ION',
  'PLEA',
  'HELP',
  'WITH',
  'OPTI

In [14]:
all_possible_tickers = []
for i in complete_ticker_list:
    all_possible_tickers.extend(i)
all_possible_tickers = list(dict.fromkeys(all_possible_tickers))
all_possible_tickers

['LIT',
 'PLT',
 'TSL',
 'FUT',
 'PLTR',
 'TSLA',
 'FUTU',
 'GME',
 'SPY',
 'WSB',
 'WSBV',
 'MGM',
 'NSA',
 'BNG',
 'BNGO',
 'THA',
 'YOU',
 'WAL',
 'LST',
 'REE',
 'TBE',
 'FOR',
 'THI',
 'AMA',
 'ZIN',
 'YEA',
 'THAN',
 'WALL',
 'STRE',
 'ETBE',
 'THIS',
 'AMAZ',
 'YEAR',
 'FSD',
 'NPA',
 'CAL',
 'CALL',
 'AST',
 'AOC',
 'YOL',
 'TRA',
 'DES',
 'WOR',
 'YOLO',
 'TRAD',
 'WORK',
 'STO',
 'POS',
 'TIN',
 'STOP',
 'POST',
 'AMC',
 'BUL',
 'NKS',
 'BULL',
 'STON',
 'JUI',
 'ICR',
 'JUIC',
 'ENP',
 'ENPH',
 'BAB',
 'BABA',
 'XOM',
 'FTW',
 'AMD',
 'NIO',
 'NYS',
 'NYSE',
 'SPA',
 'SPAC',
 'MIG',
 'CON',
 'DRO',
 'PPI',
 'MIGH',
 'CONT',
 'INUE',
 'DROP',
 'PING',
 'ETF',
 'VLD',
 'VLDR',
 'ASA',
 'ASAP',
 'PBW',
 'DNN',
 'GLU',
 'GLUE',
 'AAP',
 'DRI',
 'AAPL',
 'DRIP',
 'USA',
 'USAA',
 'SEC',
 'MVP',
 'PLE',
 'ASE',
 'HEL',
 'WIT',
 'OPT',
 'ION',
 'PLEA',
 'HELP',
 'WITH',
 'OPTI',
 'NEE',
 'GUY',
 'NEED',
 'YOUR',
 'GUYS',
 'VSL',
 'RUN',
 'CAN',
 'VSLR',
 'WIL',
 'NOT',
 'LOS',
 'TH

In [16]:
dict = {'possible_tickers': all_possible_tickers}  
df = pd.DataFrame(dict)
# saving the dataframe 
df.to_csv('possible_tickers.csv') 